In [1]:
import os
import shutil
import numpy
import pandas
import c3d

In [ ]:
#VERSIONE CHE ESEGUE UN UPSAMPLING DEI DATI
#CONSIDERO SEQUENZE DI 100 FOTOGRAMMI DI SEGUITO
#salviamo i file con indici crescenti da 0
#questo ha i seguenti vantaggi:
#possiamo generare con un xrange tutti i nomi di file
#possiamo prendere una permutazione semplicemente permutando il range
#usiamo l ordine naturale per la corrispondenza con la classe(file 3.c3d->indice 4 delle labels)

#cartella base
#qui mi aspetto dentro le dat_folders
BASE_FOLDER = "/home/luca/Scrivania/uni/magistrale/primo_semestre/prml/gait/data/"
#cartella con gli output
BASE_FOLDER_OUT = "/home/luca/Scrivania/uni/magistrale/primo_semestre/prml/gait/data/processed/"
#classi 0-3(faremo +1 al massimo)
#cartelle 0-3
#dentro le data folders ho i rispettivi contenuti degli zip(cartelle con nome cognome)
DATA_FOLDERS = ["0/","1/","2/","3/"]
#classi associate alle data_folders
CLASSES = [i for i in range(len(DATA_FOLDERS))]
#parametri di interesse(presi da ferrari)
PARAMS = ["C7","REP","RUL","RASIS","RPSIS","RCA","RGT","RLE","RCA",
         "RFM","RA","LEP","LUL","LASIS","LPSIS","LCA","LGT","LLE",
         "LFM","LA"]
NUM_PARAMS = len(PARAMS)
#grandezza campione
NUM_SAMPLES = 100
#lista classi che salveremo in un file a parte
#lista perche non ho voglia di contare tutti i file,e piu lenta ma tanto lo lanciamo solo una volta
labels = []
j = 0
for i in range(len(CLASSES)):
    #entro nella cartella classe
    folder_class = BASE_FOLDER+DATA_FOLDERS[i]
    print "start working in folder {}".format(folder_class)
    #per ogni cartella paziente estraggo i c3d
    for folder_patient in os.listdir(folder_class):
        folder_patient = folder_class+folder_patient
        #lista file
        for file_name in [file_name for file_name  in os.listdir(folder_patient) if file_name[-3:] == "c3d"]:
            #questo file ha un errore nell header!
            if file_name == "CS_preOp_sc_gait05.c3d":
                continue
            #reader per leggere il c3d
            file_name = folder_patient+"/"+file_name
            reader = c3d.Reader(open(file_name,'rb'))
            
            #for group in reader.groups:
            #    print group
            #    print reader.get(group).params
            #    print "---"
            
            #estraggo labels dei points
            params = [param.strip() for param in reader.get("POINT").get("LABELS").string_array]
            #guardo che ci siano tutti quelli del file matlab di Ferrari
            #sono 52 quelli che non l hanno
            #visto che ogni paziente ha piu prove non dovrebbe essere un problema
            if len([param for param in PARAMS if param not in params]) > 0:
                continue
            #ci sono tutti ma non so in che ordine!
            #mi servono gli indici dei param in params
            indexes = numpy.array([params.index(param) for param in PARAMS])
            #li usero per accedere ai frame
            #ogni frame ha NUM_PARAMS*5 coordinate(xyz + precisione e altro)
            points_numpy = numpy.zeros((NUM_SAMPLES,NUM_PARAMS,3))
            index =0
            for k,points,analog in reader.read_frames():                
                points_numpy[index] = points[indexes,0:3]
                index+=1
                if index == NUM_SAMPLES:
                    #se sono arrivato qui devo scrivere un file
                    labels.append(i)
                    numpy.save(BASE_FOLDER_OUT+"{}.c3d".format(j),points_numpy)
                    j+=1
                    index = 0
    print "end working in folder {}".format(folder_class)
    print "-------"

#salviamo le labels gia come one-hot labels
#cioe fila di 0 con solo un 1
labels = pandas.get_dummies(numpy.array(labels,dtype="int32")).values
numpy.save(BASE_FOLDER_OUT+"labels",labels)
print "end"